In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = 'Spiral_Dataset/training'
test_dir = 'Spiral_Dataset/testing'

# Image dimensions
img_height, img_width = 224, 224
batch_size = 32

# Data augmentation and preprocessing
#train_datagen = ImageDataGenerator(
 #   rescale=1./255,
 #   rotation_range=20,
  #  width_shift_range=0.2,
  ##  height_shift_range=0.2,
  #  shear_range=0.2,
  #  zoom_range=0.2,
  #  horizontal_flip=True,
  #  fill_mode='nearest'
#)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,       # Rotate images randomly
    width_shift_range=0.2,   # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,         # Shear transformations
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest'      # Fill missing pixels
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and testing data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 72 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=test_generator, epochs=10)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.4286 - loss: 0.9760 - val_accuracy: 0.5000 - val_loss: 0.7023
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4859 - loss: 0.8523 - val_accuracy: 0.5000 - val_loss: 0.9644
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5000 - loss: 0.9031 - val_accuracy: 0.5000 - val_loss: 0.9903
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4961 - loss: 0.9442 - val_accuracy: 0.5000 - val_loss: 0.6993
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4744 - loss: 0.7440 - val_accuracy: 0.5000 - val_loss: 0.8492
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4486 - loss: 0.8160 - val_accuracy: 0.5000 - val_loss: 0.7823
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4812 - loss: 0.7806 - val_accuracy: 0.5000 - val_loss: 0.7303
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4635 - loss: 0.7136 - val_accuracy: 0.5333 - val_loss: 0.7219
Epoch 9/10
3/3 

In [3]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

# Load pre-trained ViT model and feature extractor
model_name = 'google/vit-base-patch16-224'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=2)

# Preprocess images
def preprocess_images(images):
    return feature_extractor(images, return_tensors='pt')

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(10):
    for images, labels in train_loader:
        inputs = preprocess_images(images)
        outputs = model(**inputs)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

ModuleNotFoundError: No module named 'transformers'

In [7]:
# Evaluate ResNet model
resnet_loss, resnet_accuracy = model.evaluate(test_generator)
print(f"ResNet Test Accuracy: {resnet_accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 967ms/step - accuracy: 0.5000 - loss: 0.7327
ResNet Test Accuracy: 0.5000


In [8]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict classes for the test dataset
y_pred = model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

# Get true labels
y_true = test_generator.classes

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=['Healthy', 'Parkinson']))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

# ROC-AUC score
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC-AUC Score: {roc_auc:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.5000 - loss: 0.7327
Test Accuracy: 0.5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Classification Report:
              precision    recall  f1-score   support

     Healthy       0.50      1.00      0.67        15
   Parkinson       0.00      0.00      0.00        15

    accuracy                           0.50        30
   macro avg       0.25      0.50      0.33        30
weighted avg       0.25      0.50      0.33        30

Confusion Matrix:
[[15  0]
 [15  0]]
ROC-AUC Score: 0.4578


C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_alexnet(input_shape=(224, 224, 3), num_classes=1):
    model = Sequential()

    # Layer 1
    model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 2
    model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Layer 3
    model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))

    # Layer 4
    model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))

    # Layer 5
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(num_classes, activation='sigmoid'))  # Use 'softmax' for multi-class classification

    return model

# Build the model
alexnet_model = build_alexnet(input_shape=(224, 224, 3), num_classes=1)
alexnet_model.summary()

C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 54, 54, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 26, 26, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 26, 26, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 384)         │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 12, 12, 384)         │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 12, 12, 256)         │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 5, 5, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6400)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4096)                │      26,218,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │           4,097 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,751,105 (178.34 MB)

 Trainable params: 46,751,105 (178.34 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
alexnet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = alexnet_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 694ms/step - accuracy: 0.5078 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 8.0000e-06
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 654ms/step - accuracy: 0.4719 - loss: 0.6954 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 8.0000e-06
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 663ms/step - accuracy: 0.4422 - loss: 0.6973 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 8.0000e-06
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 639ms/step - accuracy: 0.4516 - loss: 0.6964 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 8.0000e-06
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 649ms/step - accuracy: 0.4516 - loss: 0.6961 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 1.6000e-06
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 638ms/step - accuracy: 0.4719 - loss: 0.6957 - val_accuracy: 0.5000 - val_loss: 0.6936 - learning_rate: 1.6000e-06
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 618ms/step - accuracy: 0.5000 - loss:

In [14]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Evaluate the model on the test dataset
test_loss, test_accuracy = alexnet_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predict classes for the test dataset
y_pred = alexnet_model.predict(test_generator)
y_pred_classes = (y_pred > 0.5).astype(int)

# Get true labels
y_true = test_generator.classes

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes, target_names=['Healthy', 'Parkinson']))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

# ROC-AUC score
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC-AUC Score: {roc_auc:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.5000 - loss: 0.6936
Test Accuracy: 0.5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Classification Report:
              precision    recall  f1-score   support

     Healthy       0.00      0.00      0.00        15
   Parkinson       0.50      1.00      0.67        15

    accuracy                           0.50        30
   macro avg       0.25      0.50      0.33        30
weighted avg       0.25      0.50      0.33        30

Confusion Matrix:
[[ 0 15]
 [ 0 15]]
ROC-AUC Score: 0.7333


C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\firoz\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

NameError: name 'plt' is not defined